In [8]:
import random
import sys
import os
import time
from subprocess import Popen, PIPE
from pyfaidx import Fasta

In [2]:
random.randint(75,200)

193

In [9]:
HAIRPIN_IMAGE_GENERATOR_JAR = './hairpin_image_generator/ImageCalc.jar'

In [14]:
def create_random_rna(k=200):
    return "".join(random.choices(["A","U","C","G"], k=k))


def create_rna_file(n=100, filename="sequences.fasta"):
    with open(filename, 'w') as f:
        for i in range(n):
            seq_name = f'random-seq-{i}'
            seq = create_random_rna(random.randint(70,150))
            
            f.write(f'>{seq_name}\n')
            f.write(f'{seq}\n')
                    
                


In [15]:
def generate_hairpin_images(fasta_filename, data_directory):
    
    # create directory for new hairpin images
    images_directory = data_directory + "/images"
    try:
        if not os.path.exists(images_directory):
            os.makedirs(images_directory)
    except OSError:
        print ('Error: Creating directory {}'.format(images_directory))
        sys.exit(1)

    # generate new hairpin images
    sequences = Fasta(fasta_filename)
    seq_fold_dict = {}
    for key in sequences.keys():
        sequence = str(sequences[key][:].seq)
        fold = generate_hairpin_image(sequence, key, images_directory)
        seq_fold_dict[key] = (sequence, fold)

    return seq_fold_dict

def generate_hairpin_image(sequence, sequence_identifier, output_directory):

    hairpin_image_name = output_directory + "/" + sequence_identifier + '.png'
    process = Popen(['java', '-jar', HAIRPIN_IMAGE_GENERATOR_JAR, '-o', 
    	hairpin_image_name, '-s', sequence], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()

    if not stderr.decode('utf-8'): # hairpin image was generated
        stdout = stdout.decode('utf-8')
        fold_info = stdout.split('\n')[1]
        fold = fold_info.split(':')[1].strip()
    else: # hairpin image was not generated
        msg = "Cannot generate a hairpin image for {}.\nError message: {}"
        raise Exception(msg.format(sequence, stderr.decode('utf-8'))) 
        
    return fold

In [18]:
start=time.time()
create_rna_file(n=2000)
print(time.time()-start)

0.06099700927734375


In [19]:
start = time.time()
_ = generate_hairpin_images("sequences.fasta", "newrna/")
print(time.time()-start)

540.1252851486206
